In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) #Link ke gdrive

import sys
sys.path.append('/content/gdrive/My Drive') #File dari gdrive ke colab

import os
os.chdir('/content/gdrive/My Drive') #working directory menjadi /content/gdrive/My Drive 

In [ ]:
def unique(list1):
    unique_list = [] #array kosong
     
    for x in list1:  #looping 
        if x not in unique_list: #Cek apakah ada data yang berulang atau tidak
            unique_list.append(x)
    return unique_list

In [ ]:
import numpy as np
from PIL import Image
from imutils import paths


kumpl_gmbr = [] #dataset gambar
kumpl_lbl = [] #label dataset
imgpths = paths.list_images('Dataset') #path untuk mengambil dataset

for dirimg in imgpths: #looping gambar
  gmbr = Image.open(dirimg)
  gmbrz = np.array(gmbr.resize((224,224))) / 255.0 #resize + normalisasi
  kumpl_gmbr.append(gmbrz) #memasukkan dataset yang sudah diolah ke variabel

  lbl_kls = dirimg.split(os.path.sep) [-2] #pengambilan label
  kumpl_lbl.append(lbl_kls) #mengumpulkan label ke variabel
  
kelas_asli = unique(kumpl_lbl)
kelas_asli

[]

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer() #function untuk mengubah variabel label menjadi bentuk binary
kumpl_lbl = lb.fit_transform(kumpl_lbl) #proses pengubahan
#if len(kelas_asli)==2: 
#  kumpl_lbl = np.hstack((kumpl_lbl, 1 - kumpl_lbl))

In [ ]:
from sklearn.model_selection import train_test_split

ratio_latih = 0.8 #ratio data latih
ratio_uji = 0.2 #ratio data uji
 
(latihX, ujiX, latihY, ujiY) = train_test_split(np.array(kumpl_gmbr),np.array(kumpl_lbl), test_size=ratio_uji) #Proses pemisahan, test_size menentukan banyaknya data uji

print(latihX.shape)
print(latihY.shape)
print(ujiX.shape)
print(ujiY.shape)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
i=0
banyak_lth=latihX.shape
aug=[]
for jj in range(banyak_lth[0]):
  x = latihX[jj]
  x = x.reshape((1,) + x.shape)  #Penambahan dimensi agar bisa diaugmentasi

  for aug in datagen.flow(x, batch_size=1): #proses augmentasi, memasukkan gambar ke function augmentasi
      latihX = np.append(latihX, aug ,axis=0)
      label = latihY[jj]  #Pengambilan label untuk data augmentasi
      label = label.reshape((1,) + label.shape) 
      latihY = np.append(latihY, label,axis=0)  #Memasukkan label data augment ke variabel label
      if i == 1:
        break 
      i += 1

Pembentukkan Model ResNet

In [ ]:
#Library
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from keras.initializers import glorot_uniform
from keras.models import Model

Deklarasi Function

In [ ]:
def identity_block(X, f, filters, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def ResNet50(input_shape=(224, 224, 3), classes=kelas_asli):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL 
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
ResNet = ResNet50(input_shape = (224, 224, 3), classes = kelas_asli)


In [ ]:
from tensorflow.keras.optimizers import SGD, Adam,  Nadam,  Adamax

sgd = SGD(learning_rate=0.0015) 
adam = Adam(learning_rate=0.0015)
nadam = Nadam(learning_rate=0.0015)
adamax = Adamax(learning_rate=0.0015)

In [ ]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_accuracy') > 0.9):   
          print("\nReached Target")   
          self.model.stop_training = True

In [ ]:
ResNet.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])


In [ ]:
callbacks = myCallback()
history = ResNet.fit(latihX, latihY,validation_data=(ujiX, ujiY), epochs = 55, batch_size = 32, callbacks=[callbacks]) #early stopping
#history = ResNet.fit(latihX, latihY,validation_data=(ujiX, ujiY), epochs = 55, batch_size = 32) #tanpa early stopping

In [ ]:
#evaluasi
from sklearn.metrics import classification_report

print("[INFO] evaluating network...")
predictions = ResNet.predict(ujiX, batch_size=32) 
print(classification_report(ujiY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))  

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
#Memasukkan variabel2 dari proses training, menyimpan nilai akurasi val akurasi loss dan val loss

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
#Memunculkan grafik

plt.show()

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots()
ax.plot([None] + history.history['loss'], 'o-')
ax.plot([None] + history.history['val_loss'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax.legend(['Train Loss', 'Validation Loss'], loc = 0)
ax.set_title('Training/Validation Loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')


In [ ]:
from sklearn.metrics import confusion_matrix
predict = ResNet.predict(ujiX) #Mendapatkan hasil prediksi dari model
rounded_pred=np.argmax(predict, axis=1) #Merubah dimensi
rounded_ujiY=np.argmax(ujiY, axis=1) #Merubah dimensi
print(confusion_matrix(rounded_ujiY, rounded_pred))


In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names=kelas_asli,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    if normalize:
      plt.imshow(cm/7, interpolation='nearest', cmap=cmap)
    else:
      plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.3f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
cm = confusion_matrix(rounded_ujiY,  rounded_pred)
plot_confusion_matrix(cm)

In [ ]:
FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)
print(FP)
print(FN)
print(TP)
print(TN)